In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from scipy.stats import skew, kurtosis, shapiro
plt.style.use(['bmh'])
import talib
from pandas_datareader import data as pdr
import yfinance as yfin
yfin.pdr_override()
import yaml
from utils import utils

In [ ]:
all_config = yaml.safe_load(open("../config_file.yml", "r"))

In [ ]:
df_input = utils.get_ticks_data('../src/data/example_data.csv')

In [ ]:
start_date, end_date = utils.get_start_end(all_config)
source = all_config['stocks']['source']
metric = all_config['stocks']['metric']
ticks = df_input['TICK'].to_list()

In [ ]:
end_date

In [ ]:
df = utils.get_stock_data(ticks, start_date, end_date, col=metric)

In [ ]:
df_clean = pd.DataFrame(columns=['price', 'stock', 'rsi', 'ema_12', 'ema_26', 'bb_low', 'bb_mid', 'bb_up'])
for col in df.columns.to_list():
    df_aux = pd.DataFrame()
    df_aux = df[[col]]
    df_aux.columns = ['price']
    df_aux['stock'] = col
    df_aux['rsi'] = talib.RSI(df.loc[:, col])
    df_aux['ema_12'] = talib.EMA(df.loc[:, col], 12)
    df_aux['ema_26'] = talib.EMA(df.loc[:, col], 26)
    df_aux['ema_50'] = talib.EMA(df.loc[:, col], 50)
    upper_1sd, mid_1sd, lower_1sd = talib.BBANDS(df.loc[:, col], nbdevup=1, nbdevdn=1, timeperiod=20)
    df_aux['bb_low'] = upper_1sd
    df_aux['bb_mid'] = mid_1sd
    df_aux['bb_up'] = lower_1sd
    df_clean = df_clean.append(df_aux)

In [ ]:
df_clean['rsi_f'] = np.where(df_clean['rsi'] <= 45, 1, 0)
df_clean['bb_f'] = np.where(df_clean['price'] <= df_clean['bb_low'], 1, 0)
df_clean['ema_f'] = np.where(df_clean['ema_12'] >= df_clean['ema_26'], 1, 0)
df_clean['score'] = df_clean['ema_f'] + df_clean['rsi_f'] + df_clean['bb_f']

In [ ]:
date_max = (max(df_clean.index)).strftime("%Y-%m-%d")
df_clean[(df_clean['score']>=2) & (df_clean.index==date_max)].sort_values('score', ascending=False)

### Pending

In [ ]:
df_test = df['AAPL']

In [ ]:
talib.RSI(df[]).plot(figsize=(20,5))

In [ ]:
df[['APPL']].plot(figsize=(20,8))

In [ ]:
upper_1sd, mid_1sd, lower_1sd = talib.BBANDS(df[tk], 
                                     nbdevup=1,
                                     nbdevdn=1,
                                     timeperiod=20)
# Plot the upper and lower Bollinger Bands 
plt.figure(figsize=(20,8))
plt.plot(df[tk], color='green', label='Price')
plt.plot(upper_1sd, color='tomato', label="Upper 1sd")
plt.plot(lower_1sd, color='tomato', label='Lower 2sd')

# Customize and show the plot
plt.legend(loc='upper left')
plt.title('Bollinger Bands (1sd)')
plt.show()
